In [179]:
import symd
import matplotlib.pyplot as plt
import matplotlib as mpl
import numpy as np
import pickle
import pandas as pd
import skunk
import svglib
import seaborn as sns

In [ ]:
base_colors = ["f94144","f3722c","f8961e","f9844a","f9c74f","90be6d","43aa8b","4d908e","577590","277da1"]    
colors = ['#' + c for c in base_colors]
sns.set_style("white")
sns.set_style("ticks")
sns.set(rc={'axes.facecolor':'#f5f4e9', 
            'grid.color' : '#AAAAAA', 
            'axes.edgecolor':'#333333', 
            'figure.facecolor':'#FFFFFF', 
            'axes.grid': False,
            'axes.prop_cycle':   plt.cycler('color', plt.cm.Dark2.colors),
            'font.family': 'monospace'
           })
print(symd.__version__)

In [ ]:
def run_sim(n, number_density, group, w=None, retries=5, pos_frames=0, steps=30000):
    for _ in range(retries):
        try:
            cell = symd.groups.get_cell(number_density, group, 3, n, w)
            md = symd.Symd(nparticles=n, cell=cell, ndims=3, images=2, force='lj', wyckoffs=w,
              group=group, steps=steps, exeDir='sim2d', start_temperature=0.5)
            md.remove_overlap()
            if pos_frames > 0:
                md.log_positions(frames=pos_frames)
            md.log_output(period = int(1 / md.runParams['time_step']))
            md.run()
            break
        except RuntimeError as e:
            print(e)
            md = None
    return md

In [ ]:
np.random.seed(0)
md = run_sim(5, 0.2, 17, pos_frames=100)

In [ ]:
plt.plot(md.pe, label='potential')
plt.plot(md.ke, label='kinetic')
plt.plot(md.te, label='total')
plt.legend(loc='best')

## All Sims

In [ ]:
titles = [str(i) for i in range(1,231,14)]
df = pd.DataFrame()
retries = 3
def standardize(te):
    i = int(md.te.shape[0] * 0.2)
    te = md.te[i:]
    return te - np.mean(te)

In [ ]:
for i,t in enumerate(titles):    
    md = run_sim(4, 0.2, i+1)
    df = df.assign(**{t: standardize(md.te)})    

In [ ]:
fig = plt.figure(figsize=(4,5.7))
ax = plt.gca()
mx = df.shape[0] // 2
for i,n in enumerate(df.columns):
    color = colors[i % len(base_colors)]
    ax.plot(df[n] + i, color=color)
    offsetbox = mpl.offsetbox.TextArea(n)
    ab = mpl.offsetbox.AnnotationBbox(offsetbox, (mx,i),
                    xybox=(mx,i),
                    xycoords='data',
                    boxcoords='data',
                    arrowprops=None,
                    bboxprops=dict(fc="#f5f4e9", lw=0))
    ax.add_artist(ab)
ax.set_xlabel(r'Time [$\tau$]')
ax.set_facecolor('#f5f4e9')
ax.set_ylabel(r'$\Delta$ Energy [$\epsilon$]')
plt.savefig('energy3d.svg')

## Movie

In [196]:
import moviepy.editor as editor
from moviepy.video.io.bindings import mplfig_to_npimage

def limit(traj, lim=(-5,5)):
    out = np.copy(traj)
    imin = out < lim[0]
    imax = out > lim[1]
    # set elements outside of range as nan
    out[imin] = np.nan
    out[imax] = np.nan
    # set particles with one nan element as all nan
    inan = np.any(np.isnan(out), axis=-1)
    out[inan, :] = np.nan
    return out

def plot_traj(traj, P, M, cell, title, color='#333333', fps=60):
    # set-up two subplots (cell and whole system)
    T, N, D = traj.shape        
    fps = fps
    duration = T / fps
    dpi = 90
    fig = plt.figure(figsize=(1200 / dpi, 800 / dpi), dpi=dpi)
    # make some colors for cell
    tc = [np.random.choice(colors) for _ in range(P)] * (M // P)
    # limit trajectory  to be within a cube
    cell_limits = (min(cell), max(cell))
    traj_limits = [2 * l for l in limits]
    traj2 = limit(traj, traj_limits)
    
    ax1 = fig.add_subplot(1, 2, 2, projection='3d')    
    ax2 = fig.add_subplot(1, 2, 1, projection='3d')
    
    ax1.set_xlim(*cell_limits)
    ax1.set_ylim(*cell_limits)
    ax1.set_zlim(*cell_limits)
    ax2.set_xlim(*traj_limits)
    ax2.set_ylim(*traj_limits)
    ax2.set_zlim(*traj_limits)    
    # plot cell
    scale1 = 400 / (max(cell_limits) - min(cell_limits))
    scale2 = 120 / (max(traj_limits) - min(traj_limits))
    points1 = [ax1.scatter(traj[0,:M,0], traj[0,:M,1], traj[0,:M,2], 
                          color=tc,
                          marker='o', s=scale1**2, linewidths=2, edgecolors='#999')]
    points2 = ax2.plot(traj2[0,:,0], traj2[0,:,1], traj2[0,:,2], c=color, mec='#999', marker='o',                        
                        linestyle='None', markersize=scale2)[0]
    ax2.set_facecolor('#f5f4e9')
    ax1.set_facecolor('#f5f4e9')
    fig.patch.set_facecolor('#f5f4e9')
    ax1.set_title('Unit Cell', fontsize=32, color='#333333',fontname='monospace')
    ax2.set_title(title, fontsize=32, color='#333333',fontname='monospace')
    ax1.axis('off')
    ax2.axis('off')
    angle = 60
    plt.tight_layout()
    def make_frame(t):
        i = int(t  * fps)
        i = max(0, min(i, T-1))
        points1[0].remove()
        points1[0] = ax1.scatter(traj[i,:M,0], traj[i,:M,1], traj[i,:M,2], 
                          color=tc,
                          marker='o', s=scale1**2, linewidths=1, edgecolors='#999')
        points2.set_data_3d(traj2[i,:,0], traj2[i,:,1], traj2[i,:,2])    
        #ax1.view_init(30, (angle + t * 3) % 360)
        #ax2.view_init(30, (angle + t * 3) % 360)
        plt.draw()
        return mplfig_to_npimage(fig)

    return editor.VideoClip(make_frame, duration=duration)
def write_video(clips, output, fps=60, transition=0.25):
    composite = editor.concatenate(clips[:1] + [c.crossfadein(transition) for c in clips[1:]], 
                                   padding=-transition, method='compose')   
    composite.write_videofile(output, fps=fps, preset='slower', ffmpeg_params=['-tune', 'animation'])
    
def run_nvesim(n, number_density, group, w=None, retries=5, pos_frames=0, steps=30000):
    for _ in range(retries):
        try:
            cell = symd.groups.get_cell(number_density, group, 3, n, w)
            md = symd.Symd(nparticles=n, cell=cell, ndims=3, images=2, force='lj', wyckoffs=w,
              group=group, steps=steps, exeDir='sim3d', start_temperature=0.2, temperature=None, pressure=None)
            md.remove_overlap()
            if pos_frames > 0:
                md.log_positions(frames=pos_frames)
            md.run()
            break
        except RuntimeError as e:
            print(e)
            md = None
    return md

def run_nvt(n, number_density, group, w=None, retries=5, pos_frames=0, steps=30000):
    for _ in range(retries):
        try:
            cell = symd.groups.get_cell(number_density, group, 3, n, w)
            md = symd.Symd(nparticles=n, cell=cell, ndims=3, images=2, force='lj', wyckoffs=w,
              group=group, steps=steps, exeDir='sim3d', start_temperature=0.5, temperature=0.1, thermostat='baoab')
            md.runParams['langevin_gamma'] = 1e-3
            md.remove_overlap()
            if pos_frames > 0:
                md.log_positions(frames=pos_frames)
            md.run()
            break
        except RuntimeError as e:
            print(e)
            md = None
    return md


def run_nptsim(n, number_density, group, w=None, retries=5, pos_frames=0, steps=30000):
    for _ in range(retries):
        try:
            cell = symd.groups.get_cell(number_density, group, 3, n, w)
            md = symd.Symd(nparticles=n, cell=cell, ndims=3, images=2, force='lj', wyckoffs=w,
              group=group, steps=steps, exeDir='sim3d', temperature=0.2, pressure=1, thermostat='baoab')
            md.runParams['langevin_gamma'] = 5e-2
            md.remove_overlap()
            if pos_frames > 0:
                md.log_positions(frames=pos_frames)
            md.run()
            break
        except RuntimeError as e:
            print(e)
            md = None
    return md

In [197]:
gnum = 190
md = run_nptsim(5, 0.01, gnum, pos_frames=3000, steps=180000)

{"steps": 180000, "n_particles": 5, "time_step": 0.005, "seed": 435423, "anderson_nu": 100, "langevin_gamma": 0.05, "bussi_taut": 0.5, "thermostat_seed": 54344, "rcut": 10, "skin": 0.6000000000000001, "temperature": 0.2, "pressure": 1, "box_update_period": 5, "start_temperature": 0.2, "thermostat": "baoab", "force_type": "lj", "final_positions": "sim3d/final_positions.dat", "cell_log_file": "sim3d/cell_log_file.dat", "print_period": 1800.0, "cell": [5.5603, 0.0, 0.0, 0.0, 5.17677, 0.0, 0.0, 0.0, 5.53469], "lj_epsilon": 1, "lj_sigma": 1, "n_images": 2, "start_positions": "sim3d/final_positions.dat", "group": "sim3d/group-190.json", "position_log_period": 60, "positions_log_file": "sim3d/positions.xyz"}
Info: You are running version cf97-dirty of symd
Assuming you'll pass parameters via stdin. Waiting...
Info: Loaded group group-190 with 5 particles and 12 members
Info: Duplicating 5 particles into 5 real particles and 7495 ghost for group with 12 elements and 124 tilings. Each cell has 

KeyboardInterrupt: 

In [ ]:
c = plot_traj(md.positions, md.nparticles, md.cell_nparticles, md.read_cell(), title=f'Hall Group {gnum}')
write_video([c], '3d.mp4')